In [7]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import sys
import re
import os

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Start up

In [8]:
if sys.platform == 'darwin':
    driver = webdriver.Chrome('../webdriver/chromedriver_mac')
elif sys.platform == 'win32':
    driver = webdriver.Chrome('../webdriver/chromedriver_win.exe')

In [9]:
driver.get('https://bungol.ca')

In [10]:
login = driver.find_element_by_link_text('Login/Signup')
login.click()

username = driver.find_element_by_name('username')
password = driver.find_element_by_name('password')

username.send_keys('scottw')
password.send_keys('zojzes-dycpoQ-2nidwy')
password.send_keys(Keys.RETURN)

# Gather properties

Choose an area and set filter conditions in bungol before you run the cells below

In [78]:
urls = []

leftSidebarClose = driver.find_element_by_id('leftSidebarClose')
if not ('leftSidebarClosed' in leftSidebarClose.get_attribute('class')):
    leftSidebarClose.click()

icons = driver.find_elements_by_css_selector('div.leaflet-marker-icon')
for icon in icons:
    actions = ActionChains(driver)
    actions.move_to_element(icon).perform()
    sleep(1)

    popup = driver.find_element_by_id('popup')
    urls += [a.get_attribute('href') for a in popup.find_elements_by_tag_name('a')]
    
    # Clear the popup
    driver.execute_script('arguments[0].innerHTML = ""', popup)

In [79]:
len(urls)

15

In [80]:
details = []

# 70% of aunty's home
standardExpenses = pd.Series({
    'Water': 56.21,
    'Heat': 98.41,
    'Insurance': 71.69,
    'CAC': 0,             # included in hydro
    'Hydro': 106.6
}) * 0.7

for i, url in enumerate(urls):
    if i % 10 == 0 and i > 0:
        print(f'{i} / {len(urls)}')
        
    driver.get(url)
    driver.find_element_by_link_text('Community').click()
    
    try:
        wait = WebDriverWait(driver, 5, poll_frequency=1)
        wait.until(EC.text_to_be_present_in_element((By.ID, 'listingCommunityAverageIncome'), ','))
    except:
        pass
    
    soup = BeautifulSoup(driver.page_source)
    s = pd.Series()

    # Variables whose format is <div id="#...">value</div>
    info = {
        'Address': '#listingAddress',
        'PropertyType': '#listingPropertyType',
        'AvgHouseholdIncome': '#listingCommunityAverageIncome',
        'AvgHouseholdSize': '#listingCommunityHouseholdSize'
    }
    
    for variable, selector in info.items():
        value = soup.select(selector)[0].get_text()
        s[variable] = value

    # Variables whose format is <div id="#...">label: value</div>
    info = {
        'ListPrice': '#listingListPrice',
        'ListingDate': '#listingContractDate',
        'SoldDate': '#listingEndDate',
        'SquareFootage': '#listingSummarySqFt',
        'MaintenanceFee': '#listingSummaryMaintenanceFees',
        'PropertyTax': '#listingSummaryTaxes',
        'MLS': '#listingMiscMLS',
        'DayOnMarket': '#listingDOM'
    }

    for variable, selector in info.items():
        tmp = soup.select(selector)[0].get_text().split(':')
        value = tmp[1] if len(tmp) == 2 else None
        s[variable] = value

    # Some special cases
    tmp = soup.select('#listingStatus')[0].get_text().split(' - ')
    s['Status'], s['Price'] = tmp if len(tmp) == 2 else (tmp[0], None)

    tmp = soup.select('#listingBedBath')[0].get_text().split('|')
    s['Bedrooms'], s['Bathrooms'], s['ParkingSpots'] = tmp if len(tmp) == 3 else (None, None, None)
    
    s = s.str.strip()
    
    # Monthly expense and condo fees, if applicable
    expenses = standardExpenses.copy()
    
    if driver.find_element_by_id('listingCondoFeesTable') is None:
        s['IsTaxesIncluded'] = False
        expenses['Parking'] = 0
    else:
        for item in ['Water', 'Heat', 'Insurance', 'CAC', 'Hydro', 'Parking', 'Taxes']:
            tmp = driver.find_element_by_id(f'listingCondoFeesTable{item}')
            isCovered = 'bg-success' in tmp.get_attribute('class')

            if item == 'Taxes':
                s['IsTaxesIncluded'] = isCovered
            elif isCovered:
                expenses[item] = 0

    s['UncoveredExpenses'] = expenses.sum()
    s['URL'] = url
    details.append(s)

10 / 15


In [81]:
df = pd.DataFrame(details)

In [82]:
s = df['Address'].str.extract(r'(.\d+) - (\d.+)', expand=True)
s.columns = ['Unit', 'Address']

df['Unit'] = s['Unit'].fillna('')
df['Address'] = s['Address'].combine_first(df['Address'])

In [83]:
df['PropertyType'] = df['PropertyType'].map({
    'Condo Apartment': 'Condo',
    'Townhouse (Condo)': 'Townhouse',
    'Detached House': 'Detached'
})

In [84]:
cols = ['ListingDate', 'SoldDate']
df[cols] = df[cols].apply(pd.to_datetime, axis=1)

In [85]:
cols = ['ListPrice', 'Price', 'MaintenanceFee', 'PropertyTax', 'AvgHouseholdIncome', 'AvgHouseholdSize']
df[cols] = df[cols].apply(lambda col: col.str.replace(r'[^0-9\.]', '')).apply(pd.to_numeric)

In [86]:
df['AvgPersonalIncome'] = df['AvgHouseholdIncome'] / df['AvgHouseholdSize']

The reported tax on the listing tends to be lower than it really is. We take the *greater* of the reported property tax and Toronto's property tax rate of 0.51%.

In [87]:
df['PropertyTax'] = pd.DataFrame({
    'ReportedTax': df['PropertyTax'],
    'CalculatedTax': df['ListPrice'] * 0.0051 / 12
}).max(axis=1)

In [88]:
df['MonthlyExpenses'] = df['MaintenanceFee'].fillna(0) + df['UncoveredExpenses'].fillna(0)

# If maintenance fee doesn't include property tax, add that in
cond = df['IsTaxesIncluded'] == False
df.loc[cond, 'MonthlyExpenses'] += df.loc[cond, 'PropertyTax']

df['TrueMaintenanceFee'] = df['MonthlyExpenses'] - df['PropertyTax'] - standardExpenses.sum()

In [89]:
df['Bedrooms'] = df['Bedrooms'].apply(eval)

cols = ['Bathrooms', 'ParkingSpots']
df[cols] = df[cols].apply(pd.to_numeric)

In [90]:
s = pd.to_timedelta(df['DayOnMarket'], errors='coerce') / pd.Timedelta(days=1)
t = df['DayOnMarket'].str.extract(r'(?P<Month>\d+) months?, (?P<Day>\d+) days?') \
        .astype('float') \
        .apply(lambda row: row['Month'] * 30 + row['Day'], axis=1)

df['DayOnMarket'] = s.combine_first(t).astype('int')

In [91]:
def make_hyperlink(cell):
    return f'<a target="_blank" href="{cell}">Link</a>'
    
result = df[['PropertyType', 'Unit', 'Address', 'Status', 'SquareFootage', 'ListPrice', 'MonthlyExpenses', 'TrueMaintenanceFee', 'URL', 'MLS', 'DayOnMarket']] \
            .sort_values('MonthlyExpenses').reset_index(drop=True) \
            .style.format({
                'ListPrice': '{:,.0f}',
                'PropertyTax': '{:,.2f}',
                'MonthlyExpenses': '{:,.0f}',
                'TrueMaintenanceFee': '{:,.0f}',
                'AvgPersonalIncome': '{:,.0f}',
                'URL': make_hyperlink
            })
result

,PropertyType,Unit,Address,Status,SquareFootage,ListPrice,MonthlyExpenses,TrueMaintenanceFee,URL,MLS,DayOnMarket
0,Condo,#409,"275 Yorkland Rd, North York",For Sale,800-899,"575,900",888,410,Link,C4555880,5
1,Condo,#505,"50 Forest Manor Rd, North York",For Sale,700-799,"620,000",903,407,Link,C4557425,0
2,Condo,#308,"66 Forest Manor Rd, North York",Offer Pending,700-799,"599,000",916,428,Link,C4498125,63
3,Condo,#610,"275 Yorkland Rd, North York",For Sale,800-899,"578,900",937,458,Link,C4515230,48
4,Condo,#1005,"150 Fairview Mall Dr, North York",For Sale,700-799,"629,000",938,438,Link,C4513328,48
5,Townhouse,#5,"8 Esterbrooke Ave, North York",For Sale,1200-1399,"588,000",965,482,Link,C4556310,2
6,Condo,#905,"52 Forest Manor Rd, North York",For Sale,700-799,"618,800",974,478,Link,C4528628,33
7,Condo,#1108,"5 Old Sheppard Ave, North York",For Sale,900-999,"368,000","1,005",615,Link,C4557181,0
8,Condo,#124,"52 Forest Manor Rd, North York",For Sale,800-899,"607,000","1,007",516,Link,C4555363,4
9,Townhouse,#100,"25 Esterbrooke Ave, North York",For Sale,1200-1399,"598,000","1,043",556,Link,C4554688,5


# Export

In [92]:
# result.to_excel('./result.xlsx', index=False)